## Data Cleaning

### End Goal
The goal of this notebook is to clean the data and prepare it for analysis. This includes combining the datasets, replacing N.A. values in reviews and filtering out rows where there was no rating given.

### Import Dependencies

In [24]:
import pandas as pd
import re
import numpy as np
# for detection of none english text
from langdetect import detect, DetectorFactory
# for translation purposes
from deep_translator import GoogleTranslator
# for handling emojis
import emoji

# For consistency in language detection
DetectorFactory.seed = 42

### Import the datasets

In [25]:
df_apify = pd.read_csv('../data/google_reviews_apify_dataset.csv')
df_kaggle =pd.read_csv('../data/kaggle_reviews_raw.csv')

In [27]:
# Dataset Statistics
print(f"Kaggle: {len(df_kaggle)} reviews, Apify: {len(df_apify)} reviews")
print("Kaggle columns:", df_kaggle.columns.tolist())
print("Apify columns:", df_apify.columns.tolist())

Kaggle: 1100 reviews, Apify: 1000 reviews
Kaggle columns: ['business_name', 'author_name', 'text', 'photo', 'rating', 'rating_category']
Apify columns: ['title', 'url', 'stars', 'name', 'reviewUrl', 'text']


### Standardize columns

In [28]:
df_apify_filtered = df_apify[["title", "stars", "text", "name"]].rename(columns={
    "title": "store_name",
    "stars": "rating",
    "text": "review",
    "name":"reviewer_name",
})

df_kaggle_filtered = df_kaggle[["business_name", "rating", "text", "author_name"]].rename(columns={
    "business_name": "store_name",
    "rating": "rating",
    "text": "review",
    "author_name": "reviewer_name",
})


In [30]:
print("Updated column headers for df_apify:", df_apify_filtered.columns.tolist())
print("Updated column headers for df_kaggle:", df_kaggle_filtered.columns.tolist())

Updated column headers for df_apify: ['store_name', 'rating', 'review', 'reviewer_name']
Updated column headers for df_kaggle: ['store_name', 'rating', 'review', 'reviewer_name']


### Combining the datasets

In [31]:
df_combined = pd.concat([df_apify_filtered, df_kaggle_filtered], ignore_index=True)

In [32]:
df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2100 entries, 0 to 2099
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   store_name     2100 non-null   object
 1   rating         2100 non-null   int64 
 2   review         1845 non-null   object
 3   reviewer_name  2100 non-null   object
dtypes: int64(1), object(3)
memory usage: 65.8+ KB


### Replacing Rows with no reviews with "No reviews"

In [33]:
df_combined["review"] = df_combined["review"].replace("", "No reviews").replace(np.nan, "No reviews")

In [34]:
# confirming no missing values
df_combined["review"].isna().any()

np.False_

### Handling of emojis in reviews

In [35]:
def remove_emojis(review):
    if emoji.emoji_count(review) > 0:
        # Replace emojis with their text representation
        review = emoji.replace_emoji(review, '')
    return review

df_combined["review"] = df_combined["review"].apply(remove_emojis)

In [36]:
# check for presence of emojis in reviews
df_combined["review"].apply(emoji.emoji_count).sum()

np.int64(0)

In [41]:
df_combined["review"].replace('', 'No reviews', inplace=True)

Number of empty reviews after removing emojis

In [42]:
count = 0
for each in df_combined["review"]:
    if (each == ''):
        print("Empty review detected")
        count += 1
print(f"Total empty reviews detected: {count}")

Total empty reviews detected: 0


In [43]:
df_combined

,store_name,rating,review,reviewer_name
0,49 SEATS,5,No reviews,yeong kx
1,49 SEATS,5,wowowow great vibes and food!! super eccentric...,Hannah Eva
2,49 SEATS,5,No reviews,Rain
3,49 SEATS,5,No reviews,Nicolette David
4,49 SEATS,4,We had the classic pasta and fish n chips with...,S dssp
...,...,...,...,...
2095,Miss Pizza,5,There are so many types of pizza; you are surp...,Salih Gursoy
2096,Miss Pizza,5,I tried the smoked ribeye pizza; the dough is ...,Kemal Amangeldi
2097,Miss Pizza,3,Crowded and expensive place.,Ulkem Esen
2098,Miss Pizza,3,No bad. It was very crowded; there was no ligh...,Ilkin Saymaz


### Translation for non-english reviews

In [44]:
df_combined["review"] = df_combined["review"].str.replace("\n", " ").str.replace("/", " ")

def detect_and_translate(review):
    language = detect(review)
    if language != 'en':
        try:
            translated_review = GoogleTranslator(source='auto', target='en').translate(review)
            return translated_review
        except Exception as e:
            print(f"Translation error for review: {review}. Error: {e}")
            return review
    else:
        return review

df_combined_translated = df_combined.copy()
df_combined_translated["review"] = df_combined_translated["review"].apply(detect_and_translate)

### Remove entries which are still in other languages


In [45]:
df_combined_translated = df_combined_translated[
    df_combined_translated["review"].apply(lambda x: detect(str(x)) == "en")
].reset_index(drop=True)


In [46]:
count = 0
for review in df_combined_translated["review"]:
    if detect(review) != 'en':
        print("Non-English review found after translation:", review)
        count += 1
print("Total non-English reviews found:", count)

Total non-English reviews found: 0


### Remove linespaces and make the sentence lowercase

In [49]:
# Clean reviews: remove newlines, slashes, extra spaces, lowercase
def clean_review(text):
    if isinstance(text, str):
        text = text.replace("\n", " ")       # replace line breaks with space
        text = text.replace("/", " ")        # replace slashes with space
        text = re.sub(r"\s+", " ", text)     # collapse multiple spaces into one
        text = text.strip().lower()          # strip leading/trailing spaces + lowercase
    return text

df_combined_translated["review"] = df_combined_translated["review"].apply(clean_review)

### Generate cleaned and updated file

In [50]:
pd.DataFrame(df_combined_translated).to_csv('../data/cleaned_reviews.csv', index=False)

In [51]:
df_combined_translated.shape

(2032, 4)